In [1]:
!pip install psutil

You should consider upgrading via the '/Users/melanie/Dev/whylogs-v1/python/.venv/bin/python -m pip install --upgrade pip' command.


In [9]:
import time
import psutil
import whylogs as why

In [10]:
utility_details = {}

with why.logger(mode="rolling", interval=5, when="S", base_name="test_base_name") as logger:
    while True:
        utility_details = {"time": time.gmtime(), "cpu_percentage": psutil.cpu_percent()}
        logger.log(utility_details)
        print(utility_details)
        time.sleep(1)

{'time': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=1, tm_hour=20, tm_min=20, tm_sec=20, tm_wday=2, tm_yday=152, tm_isdst=0), 'cpu_percentage': 12.8}
{'time': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=1, tm_hour=20, tm_min=20, tm_sec=21, tm_wday=2, tm_yday=152, tm_isdst=0), 'cpu_percentage': 24.1}
{'time': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=1, tm_hour=20, tm_min=20, tm_sec=22, tm_wday=2, tm_yday=152, tm_isdst=0), 'cpu_percentage': 22.7}
{'time': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=1, tm_hour=20, tm_min=20, tm_sec=23, tm_wday=2, tm_yday=152, tm_isdst=0), 'cpu_percentage': 20.7}
{'time': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=1, tm_hour=20, tm_min=20, tm_sec=24, tm_wday=2, tm_yday=152, tm_isdst=0), 'cpu_percentage': 24.6}
{'time': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=1, tm_hour=20, tm_min=20, tm_sec=25, tm_wday=2, tm_yday=152, tm_isdst=0), 'cpu_percentage': 24.4}
{'time': time.struct_time(tm_year=2022, tm_mon=6, tm_mday=1, tm_

KeyboardInterrupt: 

In [ ]:
logger.